**GIỚI THIỆU VỀ TẬP DỮ LIỆU**

Quora là một nền tảng cho phép mọi người học hỏi lẫn nhau. Trên Quora, mọi người có thể đặt câu hỏi và kết nối với những người khác, những người đóng góp thông tin chi tiết độc đáo và câu trả lời chất lượng. Một thách thức quan trọng là loại bỏ những câu hỏi thiếu chân thành - những câu hỏi được đặt ra dựa trên những tiền đề sai lầm hoặc có ý định đưa ra một tuyên bố hơn là tìm kiếm những câu trả lời hữu ích.

Trong cuộc thi này, Kagglers sẽ phát triển các mô hình xác định và gắn cờ cho các câu hỏi không chân thành.

**Mô tả tệp**

* train.csv - tập huấn luyện
* test.csv - bộ thử nghiệm


**Các trường dữ liệu**

* qid - mã định danh câu hỏi duy nhất
* question_text - câu hỏi Quora
* target - câu hỏi có nhãn "insincere" có giá trị bằng 1, ngược lại bằng 0

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import glob

# DATA OVERVIEW

**ĐỌC DỮ LIỆU**

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

In [ ]:
train.info()

***Nhận xét:*** Dữ liệu huấn luyện không có giá trị null

In [ ]:
test.info()

***Nhận xét:*** Dữ liệu kiểm thử không có giá trị null

**TỈ LỆ PHÂN BỐ NHÃN TRONG TẬP DỮ LIỆU HUẤN LUYỆN**

In [ ]:
ax, fig = plt.subplots(figsize=(10, 7))
question_class = train["target"].value_counts()
question_class.plot(kind= 'bar', color= ["blue", "orange"])
plt.title('Bar chart')
plt.show()

In [ ]:
print("Tỉ lệ phần trăm số câu hỏi Insincere là:", (len(train.loc[train.target==1])) / (len(train.loc[train.target == 0])) * 100)

***Nhận xét***: Số câu hỏi "insincere" chỉ chiếm khoảng 6-7% trong tổng số câu hỏi. Dữ liệu bị mất cân bằng khá lớn, do đó độ đo F1 có vẻ thích hợp cho những trường hợp như này

**PHÂN TÍCH TỪNG CÂU HỎI**

**Số lượng từ trong câu**

In [ ]:
words = train['question_text'].apply(lambda x: len(x) - len(''.join(x.split())) + 1)
train['words'] = words
words = train.loc[train['words']<200]['words']
sns.distplot(words, color='g')
plt.show()

In [ ]:
print('Số lượng từ trung bình của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.mean(train['question_text'].apply(lambda x: len(x.split())))))
print('Số lượng từ trung bình của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.mean(test['question_text'].apply(lambda x: len(x.split())))))

In [ ]:
print('Số lượng từ lớn nhất của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.max(train['question_text'].apply(lambda x: len(x.split())))))
print('Số lượng từ lớn nhất của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.max(test['question_text'].apply(lambda x: len(x.split())))))

In [ ]:
print('Số lượng ký tự trung bình của các câu hỏi trong dữ liệu huấn luyện là {0:.0f}.'.format(np.mean(train['question_text'].apply(lambda x: len(x)))))
print('Số lượng ký tự trung bình của các câu hỏi trong dữ liệu kiểm thử là {0:.0f}.'.format(np.mean(test['question_text'].apply(lambda x: len(x)))))

***Nhận xét:*** Có thể thấy độ dài trung bình của các câu hỏi trong tập dữ liệu huấn luyện và kiểm thử tương tự nhau, tuy nhiên có những câu hỏi khá dài trong tập dữ liệu huấn luyện

# DATA PREPROCESSING

**Vấn đề:** Như đã phân tích về dữ liệu ở trên, ta sẽ quy chuẩn số lượng từ trong một câu, độ dài của vector sau khi chuẩn hoá câu.

In [ ]:
embed_size = 300 #độ dài của vector
max_features = 100000 #số lượng từ xuất hiện nhiều nhất để huấn luyện
maxlen = 80 #số lượng từ trong câu 

**Ở notebook này, ta sẽ không thực hiện tiền xử lý dữ liệu mà chia luôn dữ liệu để huấn luyện**

In [ ]:
## split to train and val
train, val = train_test_split(train, test_size=0.1, random_state=1023)

**Thay các giá trị còn thiếu bằng 'na'**

In [ ]:
# fill up the missing values
train_X = train["question_text"].fillna("_na_").values
val_X = val["question_text"].fillna("_na_").values
test_X = test["question_text"].fillna("_na_").values

**Tokenize các câu hỏi và chuyển thành chuỗi vector**

In [ ]:
# Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

**Pad chuỗi** - nếu số lượng từ trong câu hỏi lớn hơn 'max_len' thì chuyển thành 'max_len' hoặc nếu số từ trong văn bản ít hơn 'max_len' thì bổ sung thêm số 0 vào các giá trị còn lại

In [ ]:
# Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [ ]:
# Get the target values
train_y = train['target'].values
val_y = val['target'].values

# MODELING

**Thiết lập model**

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

**Thiết lập model**

In [ ]:
def train_pred(model, train_X, train_y, val_X, val_y, epochs=2):
    for e in range(epochs):
        model.fit(train_X, train_y, batch_size=512, epochs=1, validation_data=(val_X, val_y))
        pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)

        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
            if score > best_score:
                best_thresh = thresh
                best_score = score

        print("Val F1 Score: {:.4f}".format(best_score))

    pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    print('='*100)
    return pred_val_y, pred_test_y, best_score

**Train model**

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))

In [ ]:
pred_val_y = model.predict([val_X], batch_size=1024, verbose=0)
pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)

**Xác định threshold có F1 cao nhất**

In [ ]:
def f1_smart(y_true, y_pred):
    thresholds = []
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        res = f1_score(y_true, (y_pred > thresh).astype(int))
        thresholds.append([thresh, res])
        print("F1 score at threshold {0} is {1}".format(thresh, res))

    thresholds.sort(key=lambda x: x[1], reverse=True)
    best_thresh = thresholds[0][0]
    best_f1 = thresholds[0][1]
    print("Best threshold: ", best_thresh)
    return  best_f1, best_thresh

In [ ]:
f1, threshold = f1_smart(val_y, pred_val_y)
print('Optimal F1: {} at threshold: {}'.format(f1, threshold))

**Nhận xét:** F1 score cao nhất = 0.64 với threshold = 0.37

**Tạo submission**

In [ ]:
pred_test_y = (pred_test_y >threshold).astype(int)
out_df = pd.DataFrame({"qid":test["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)